In [ ]:
import numpy as np
import xarray as xr

In [ ]:
class Subset:
    pass


class Slice(Subset):
    slice: slice


class IndexSet(Subset):
    values: [int]


class Dynamic(Subset):
    pass


class SubsetType:
    base: DimType
    subset: list[Subset]

In [ ]:
x_sub, x_sub2 = px.project_to_shared_dim(x[:-1], x[1:], dim=foo)

In [ ]:
foo = px.dim()
x = px.xtensor(foo)

# Example 1
x[:-1].pad_to_dim(foo, fill_value=0.0)

# example 2
left_part, right_part = foo.intersect_and_align(x[:-1], x[1:])
left_part + right_part

# example 3
x[1:] + x[:-1].with_dims_like(x[1:])

In [113]:
import pytensor
import pytensor.tensor as pt
import pytensor.xtensor as px
import pytest

/home/adr/git/pytensor/pytensor/xtensor/__init__.py:18: UserWarning: xtensor module is experimental and full of bugs
  warnings.warn("xtensor module is experimental and full of bugs")


In [116]:
foo = px.dim("foo")
foo.type

BasicDim(foo, uuid=?)

In [3]:
foo = px.dim("foo")
x = px.ones(foo, name="x")
func = pytensor.function([foo], [x.sum(foo)], mode="FAST_RUN")
func(5)

[array(5.)]

In [4]:
foo = px.dim("foo")
x = px.ones(foo, name="x")
func = pytensor.function([foo], [x.std(foo)], mode="FAST_RUN")
func(5)

[array(0.)]

In [5]:
length = pytensor.scalar.basic.int64()
bar = px.dim("bar", size=length)
bar.dprint();

FromLength{dim_type=BasicDim(bar, uuid=?)} [id A] 'bar'
 └─ TensorFromScalar [id B]
    └─ <int64> [id C]


In [6]:
bar = px.dim("bar")
bar2 = px.dim("bar2", size=bar)
# same as bar.clone_dim("bar2")
bar2.dprint();

Clone{dim_type=CloneDim(bar2, base=BasicDim(bar, uuid=?), uuid=?)} [id A] 'bar2'
 └─ bar [id B]


In [7]:
foo = px.dim("foo")
bar = foo.clone_dim("bar")
x = px.xtensor("x", dims=[foo, bar])
x.dims

(foo, bar)

In [8]:
foo = px.dim("foo")
bar = px.dim("bar")
z1 = px.ones(foo)
z2 = px.ones(bar)
func = pytensor.function([foo, bar], [z1 + z2])
func(3, 5)

[array([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]])]

In [9]:
import numpy as np

In [10]:
foo = px.dim("foo")
bar = px.dim("bar")
z_tensor = pt.matrix()
z_new = px.xtensor_from_tensor(z_tensor, [foo, bar], check=True)
z_new.dprint()
func = pytensor.function([z_tensor, foo, bar], [z_new])
with pytest.raises(AssertionError):
    func(np.zeros((3, 4)), 3, 5)
func(np.zeros((3, 4)), 3, 4)

XTensorFromTensor [id A]
 ├─ SpecifyShape [id B]
 │  ├─ <Matrix(float64, shape=(?, ?))> [id C]
 │  ├─ Length [id D]
 │  │  └─ foo [id E]
 │  └─ Length [id F]
 │     └─ bar [id G]
 ├─ foo [id E]
 └─ bar [id G]


[array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])]

In [11]:
foo = px.dim("foo")
with pytest.raises(ValueError):
    px.xtensor("z", dims=[foo, foo])

In [134]:
def tensorize(inputs, outputs, *, check=True):
    dims = {}
    for input in inputs:
        if isinstance(input, px.type.DimVariable):
            dims[input.type] = input

    new_inputs = []
    replacements = []
    for input in inputs:
        if isinstance(input, px.type.DimVariable):
            replacements.append((input, input))
            new_inputs.append(input)
        else:
            new_input = px.basic.tensor_from_xtensor(input).type()
            replacement = px.xtensor_from_tensor(new_input, [dims[dim.type] for dim in input.dims], check=True)
            replacements.append((input, replacement))
            new_inputs.append(new_input)

    new_outputs = pytensor.clone_replace( outputs, replacements)
    #new_inputs = [new_input for _, new_input in replacements]
    return new_inputs, new_outputs

In [140]:
country = px.dim("country")
treatment = px.dim("treatment")

effect = px.xtensor("effect", dims=[country, treatment])
sigma = px.xtensor("sigma", dims=[])
observed = px.xtensor("observed", dims=[treatment, country])

residual = ((effect - observed) / sigma) + (effect - observed).std()

In [142]:
inputs = [country, treatment, effect, sigma, observed]
outputs = [residual]

tensor_inputs, tensor_outputs = tensorize(inputs=inputs, outputs=outputs, check=True)
func = pytensor.function(tensor_inputs, tensor_outputs)